# 1. the simpler option


In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

def tomorrows_flight_arrivals(icao_list):
  # Get today's date in Berlin timezone
  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  # Calculate tomorrow's date
  tomorrow = (today + timedelta(days=1))

  # Initialize an empty list to store flight data
  list_for_df = []

  # Loop over each ICAO code in the input list
  for icao in icao_list:
    # Define the two time periods for which to fetch data
    times = [["00:00","11:59"],["12:00","23:59"]]

    # Loop over each time period
    for time in times:
      # Construct the URL for the API request
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      # Define the query parameters for the API request
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      # Define the headers for the API request
      headers = {
          'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
          'x-rapidapi-key': "YOUR_API_KEY"
          }
      # Make the API request
      response = requests.request("GET", url, headers=headers, params=querystring)
      # Parse the JSON response
      flights_json = response.json()

      # Loop over each flight in the response
      for flight in flights_json['arrivals']:
        # Initialize an empty dictionary to store flight data
        flights_dict = {}
        # Store the ICAO code and flight data in the dictionary
        flights_dict['arrival_icao'] = icao
        # Use the .get() method to avoid KeyError if a key doesn't exist in the dictionary
        flights_dict['arrival_time_local'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_number'] = flight.get('number', None)
        # Store the current date in Berlin timezone
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        # Append the flight dictionary to the list
        list_for_df.append(flights_dict)

  # Convert the list of flight dictionaries to a DataFrame and return it
  return pd.DataFrame(list_for_df)


In [ ]:
icaos = ['EDDB', 'EGLL']
tomorrows_flight_arrivals(icaos).head()

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2023-07-06 05:45+02:00,1,İzmir,LTBJ,2023-07-06 03:40+03:00,SunExpress,XQ 966,2023-07-05
1,EDDB,2023-07-06 06:15+02:00,1,Reykjavik,BIKF,2023-07-06 00:45+00:00,Icelandair,FI 518,2023-07-05
2,EDDB,2023-07-06 06:20+02:00,1,Marsa Alam,HEMA,2023-07-06 02:20+03:00,Air Cairo,SM 2962,2023-07-05
3,EDDB,2023-07-06 06:40+02:00,2,Varna,LBWN,2023-07-06 05:10+03:00,Wizz Air,W6 4515,2023-07-05
4,EDDB,2023-07-06 07:05+02:00,1,New York,KJFK,2023-07-05 16:50-04:00,Delta Air Lines,DL 92,2023-07-05


# the more complicated option
most of you won't be making functions like this yet, but I want to leave it here to show haw things advancce and grow. This function is the same general idea as the function above, but we've tidied up the dictionary creation and added a function for error handling

In [ ]:
import requests
from datetime import datetime, timedelta
from pytz import timezone
import pandas as pd

# Helper function to make API request and handle exceptions
def get_flight_data(icao, time, date):
    # Construct the URL for the API request
    url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{date}T{time[0]}/{date}T{time[1]}"
    # Define the query parameters for the API request
    query_params = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
    # Define the headers for the API request
    headers = {
        'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
        'x-rapidapi-key': "YOUR_API_KEY"
    }

    # Make the API request and handle potential exceptions
    try:
        response = requests.get(url, headers=headers, params=query_params)
        response.raise_for_status()  # Raises stored HTTPError, if one occurred.
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
        return []
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
        return []
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
        return []
    except requests.exceptions.RequestException as err:
        print("Ops: Something went wrong", err)
        return []

    # Return the 'arrivals' data from the response JSON
    return response.json()['arrivals']

def tomorrows_flight_arrivals(icao_list):
    # Get today's date in the 'Europe/Berlin' timezone
    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    # Calculate tomorrow's date
    tomorrow = (today + timedelta(days=1))

    # Initialize an empty list to store flight data
    list_for_df = []

    # Loop over each ICAO code in the input list
    for icao in icao_list:
        # Define the two time periods for which to fetch data
        time_periods = [["00:00", "11:59"], ["12:00", "23:59"]]

        # Loop over each time period
        for time in time_periods:
            # Fetch flight data for the given ICAO code and time period
            flights = get_flight_data(icao, time, tomorrow)

            # Generate flight dictionary for each flight
            for flight in flights:
                # Extract relevant data from the flight data and store it in a dictionary
                flights_dict = {
                    'arrival_icao': icao,
                    'arrival_time_local': flight['arrival'].get('scheduledTimeLocal', None),
                    'arrival_terminal': flight['arrival'].get('terminal', None),
                    'departure_city': flight['departure']['airport'].get('name', None),
                    'departure_icao': flight['departure']['airport'].get('icao', None),
                    'departure_time_local': flight['departure'].get('scheduledTimeLocal', None),
                    'airline': flight['airline'].get('name', None),
                    'flight_number': flight.get('number', None),
                    'data_retrieved_on': datetime.now().astimezone(timezone('Europe/Berlin')).date()
                }

                # Append the flight dictionary to the list
                list_for_df.append(flights_dict)

    # Convert the list of flight dictionaries to a DataFrame and return it
    return pd.DataFrame(list_for_df)


In [ ]:
tomorrows_flight_arrivals(icaos).head()

,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EDDB,2023-07-06 05:45+02:00,1,İzmir,LTBJ,2023-07-06 03:40+03:00,SunExpress,XQ 966,2023-07-05
1,EDDB,2023-07-06 06:15+02:00,1,Reykjavik,BIKF,2023-07-06 00:45+00:00,Icelandair,FI 518,2023-07-05
2,EDDB,2023-07-06 06:20+02:00,1,Marsa Alam,HEMA,2023-07-06 02:20+03:00,Air Cairo,SM 2962,2023-07-05
3,EDDB,2023-07-06 06:40+02:00,2,Varna,LBWN,2023-07-06 05:10+03:00,Wizz Air,W6 4515,2023-07-05
4,EDDB,2023-07-06 07:05+02:00,1,New York,KJFK,2023-07-05 16:50-04:00,Delta Air Lines,DL 92,2023-07-05
